In [97]:
# DO ANY PASSES OVERLAP

In [98]:
# Import necessary libraries
import ephem
import math
from datetime import timedelta
from datetime import datetime


# get_passes() function definition
def get_passes(observer, tle, start_time, num_passes=None, duration=None):
    """Config obs and sat, Return pass data for all passes in given interval.

    Arguments:
    observer -- 4 element list containing desired [name,lat,lon,alt]
    tle -- 3 element list containing desired tle [line0,line1,line2]
    start_time -- ephem.date string formatted 'yyyy/mm/dd hr:min:sec'
    num_passes -- integer number of desired passes (defualt None)
    duration -- float number of hours or fraction of hours (default None)

    Specify either num_passes or duration.
    If both, use num_passes.
    If neither, find passes for next 24 hours.
    """

    obs_name, obs_lat, obs_lon, obs_alt = observer
    tle_line0, tle_line1, tle_line2 = tle

    # Set up location of observer
    ground_station = ephem.Observer()
    ground_station.name = obs_name                # name string
    ground_station.lon = obs_lon                  # in degrees (+E)
    ground_station.lat = obs_lat                  # in degrees (+N)
    ground_station.elevation = obs_alt            # in meters
    ground_station.date = ephem.date(start_time)  # in UTC

    # Read in most recent satellite TLE data
    sat = ephem.readtle(tle_line0, tle_line1, tle_line2)

    contacts = []

    if num_passes is not None and duration is None:
        # if only num_passes specified
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(days = set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
                contacts.append(pass_data)
        except ValueError:
            # No (more) visible passes
            pass
        return contacts

    if num_passes is None and duration is not None:
        # if only duration specified
        try:
            end_time = ephem.date(ground_station.date+duration*ephem.hour)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
                contacts.append(pass_data)
        except ValueError:
            # No (more) visible passes
            pass
        return contacts

    if num_passes is not None and duration is not None:
        # if both are specified, use num_passes
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
                contacts.append(pass_data)
        except ValueError:
            # No (more) visible passes
            pass
        return contacts

    if num_passes is None and duration is None:
        # if neither are specified, get passes for the next 24 hours
        try:
            end_time = ephem.date(ground_station.date+1)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
                contacts.append(pass_data)
        except ValueError:
            # No (more) visible passes
            pass
        return contacts


In [99]:
from itertools import islice
data = []
with open('amateur.txt') as f:
    while True:
        #an iterator that returns the next N lines and stops
        tripleline = islice(f, 3)
        #loop over these N lines, removing trailing spaces and \n
        tle = [x.rstrip() for x in tripleline]

        #only accept complete data
        #the end of the file *should* have len(tle)==0 but
        #this also handles extra junk at the end
        if len(tle) == 3:
            data.append(tle)
        else:
            break


In [104]:
vu = ['Valparaiso University', '41.4639', '-87.0439', 245.089]
start_time = '2017/6/8 00:00:00'
num_passes = None
duration = 24.0
vu_passes = {}

for tle in data:
    # use NORAD ID as key for each satellite
    # value is list of passes, where each pass is a dictionary of data
    noradID = tle[2][2:7]
    vu_passes[noradID] = get_passes(vu, tle, start_time, num_passes=num_passes, duration=duration)

    

In [111]:
a = [len(data),int(duration)]
a

[74, 24]

In [105]:
for sat, passes in vu_passes.items():
    for obs in passes:
        print('NORAD ID:', sat, 'START:', obs['start'], 'END:', obs['end'])


NORAD ID: 22826 START: Thu Jun  8 07:04:59 2017 END: Thu Jun  8 07:09:22 2017
NORAD ID: 22826 START: Thu Jun  8 08:40:31 2017 END: Thu Jun  8 08:55:26 2017
NORAD ID: 22826 START: Thu Jun  8 10:20:08 2017 END: Thu Jun  8 10:34:30 2017
NORAD ID: 22826 START: Thu Jun  8 12:02:07 2017 END: Thu Jun  8 12:08:08 2017
NORAD ID: 22826 START: Thu Jun  8 18:22:46 2017 END: Thu Jun  8 18:32:55 2017
NORAD ID: 22826 START: Thu Jun  8 19:58:38 2017 END: Thu Jun  8 20:13:46 2017
NORAD ID: 22826 START: Thu Jun  8 21:39:19 2017 END: Thu Jun  8 21:52:43 2017
NORAD ID: 22826 START: Fri Jun  9 08:11:31 2017 END: Fri Jun  9 08:25:16 2017
NORAD ID: 32789 START: Thu Jun  8 01:17:11 2017 END: Thu Jun  8 01:26:55 2017
NORAD ID: 32789 START: Thu Jun  8 02:50:04 2017 END: Thu Jun  8 03:02:31 2017
NORAD ID: 32789 START: Thu Jun  8 04:28:19 2017 END: Thu Jun  8 04:35:15 2017
NORAD ID: 32789 START: Thu Jun  8 14:27:17 2017 END: Thu Jun  8 14:38:00 2017
NORAD ID: 32789 START: Thu Jun  8 16:01:35 2017 END: Thu Jun  8 


NORAD ID: 37839 START: Thu Jun  8 05:58:59 2017 END: Thu Jun  8 06:09:15 2017
NORAD ID: 37839 START: Thu Jun  8 07:50:20 2017 END: Thu Jun  8 07:53:37 2017
NORAD ID: 37839 START: Fri Jun  9 00:23:26 2017 END: Fri Jun  9 00:27:03 2017
NORAD ID: 39440 START: Thu Jun  8 00:36:01 2017 END: Thu Jun  8 00:49:18 2017
NORAD ID: 39440 START: Thu Jun  8 02:14:12 2017 END: Thu Jun  8 02:25:07 2017
NORAD ID: 39440 START: Thu Jun  8 12:26:39 2017 END: Thu Jun  8 12:38:42 2017
NORAD ID: 39440 START: Thu Jun  8 14:02:34 2017 END: Thu Jun  8 14:16:45 2017
NORAD ID: 39440 START: Thu Jun  8 15:40:33 2017 END: Thu Jun  8 15:50:01 2017
NORAD ID: 39440 START: Thu Jun  8 23:25:53 2017 END: Thu Jun  8 23:36:30 2017
NORAD ID: 39440 START: Fri Jun  9 01:00:31 2017 END: Fri Jun  9 01:13:56 2017
NORAD ID: 41789 START: Thu Jun  8 01:32:42 2017 END: Thu Jun  8 01:44:46 2017
NORAD ID: 41789 START: Thu Jun  8 03:08:29 2017 END: Thu Jun  8 03:22:39 2017
NORAD ID: 41789 START: Thu Jun  8 04:49:37 2017 END: Thu Jun  8

END: Thu Jun  8 09:40:08 2017
NORAD ID: 39134 START: Thu Jun  8 11:15:02 2017 END: Thu Jun  8 11:16:38 2017
NORAD ID: 39134 START: Thu Jun  8 12:53:16 2017 END: Thu Jun  8 12:59:10 2017
NORAD ID: 39134 START: Thu Jun  8 14:29:58 2017 END: Thu Jun  8 14:41:11 2017
NORAD ID: 39134 START: Thu Jun  8 16:07:14 2017 END: Thu Jun  8 16:19:47 2017
NORAD ID: 39134 START: Thu Jun  8 17:46:39 2017 END: Thu Jun  8 17:53:44 2017
NORAD ID: 39134 START: Fri Jun  9 05:57:14 2017 END: Fri Jun  9 06:08:39 2017
NORAD ID: 39444 START: Thu Jun  8 00:37:12 2017 END: Thu Jun  8 00:49:18 2017
NORAD ID: 39444 START: Thu Jun  8 02:12:39 2017 END: Thu Jun  8 02:25:49 2017
NORAD ID: 39444 START: Thu Jun  8 03:54:43 2017 END: Thu Jun  8 03:58:21 2017
NORAD ID: 39444 START: Thu Jun  8 12:27:05 2017 END: Thu Jun  8 12:31:49 2017
NORAD ID: 39444 START: Thu Jun  8 13:59:59 2017 END: Thu Jun  8 14:13:31 2017
NORAD ID: 39444 START: Thu Jun  8 15:36:30 2017 END: Thu Jun  8 15:48:55 2017
NORAD ID: 39444 START: Thu Jun  8 

In [107]:
#import matplotlib.pyplot as plt
#import numpy as np
#t = np.arange(0., 5., 0.2)
#plt.plot(t,t,'r--',t,t**2,'bs',t,t**3,'g^')
#plt.axis([0,6,0,7])
#plt.ylabel('y-axis')
#plt.xlabel('x-axis')
#plt.title('Graph')
#plt.show()

In [149]:
#C_j
import scipy.integrate as integrate
T = 1
n = 10
cj = 0

def a(t):
    a = [0,0,0,0]
    return a[t]

def r(t):
    return 1

def l(t):
    return 1

def nu(t):
    return 1

def integrand(t):
    return a(t)*r(t)*l(t)*nu(t)

for i in range(n):
    cint = integrate.quad(integrand, 0, T)

TypeError: list indices must be integers or slices, not float